In [3]:
# set up imports
import pandas as pd, numpy as np, seaborn as sns
from tabulate import tabulate
import statsmodels
import arch
import matplotlib
matplotlib.use('qt5agg')

# configure plot style
import matplotlib.pyplot as plt
plt.rcParams['mathtext.fontset'] = 'stix'
plt.rcParams['font.family'] = 'STIXGeneral'
plt.rcParams["figure.figsize"] = (9.5,4.15)
plt.rcParams['figure.constrained_layout.use'] = False
plt.rcParams['figure.dpi'] = 100
plt.rcParams['savefig.dpi'] = 10
plt.rcParams['lines.linewidth'] = 0.8
save_plot_to =  r'C:\\Users\joche\OneDrive\03 TUM - TUM-BWL\Semester 8\01 Bachelorarbeit\04 Results\Plots/'

In [4]:
# load data and crop to start_date:end_date
start_date = '2015-08-07'
end_date = '2020-06-26'
idx = pd.date_range(start_date, end_date)
index_name = 'date'
mydateparser = lambda x: pd.datetime.strptime(str(x), '%Y-%m-%d %H:%M:%S')
mydateparser1 = lambda x: pd.datetime.strptime(str(x), '%Y-%m-%d')

# btc
btc = pd.read_excel('Data/BTC_closing.xlsx', parse_dates=[0], index_col=0, date_parser=mydateparser)[start_date:end_date]
btc.index.name = index_name
btc.columns = ['btc']

# usd_eur
usd_eur = pd.read_excel('Data/DEXUSEU.xls', parse_dates=[0], index_col=0, skiprows=10, date_parser=mydateparser)[start_date:end_date]
usd_eur.index.name = index_name
usd_eur.columns = ['usd_eur']
usd_eur = usd_eur.loc[(usd_eur!=0).any(1)]

# tot_btc: only weekly data - missing values interpolated
tot_btc = pd.read_csv('Data/total-bitcoins.txt', index_col=0)[start_date:end_date]
tot_btc.index = pd.DatetimeIndex(tot_btc.index, normalize=True).normalize()
tot_btc.index.name = index_name
tot_btc = tot_btc.reindex(idx, fill_value=None)
tot_btc.interpolate(method='time', inplace=True, limit_direction='both')
tot_btc.columns = ['tot_btc']

# hs_rate: only weekly data - missing values interpolated
hs_rate = pd.read_csv('Data/hash-rate.txt', index_col=0)[start_date:end_date]
hs_rate.index = pd.DatetimeIndex(hs_rate.index, normalize=True).normalize()
hs_rate.index.name = index_name
hs_rate = hs_rate.reindex(idx, fill_value=None)
hs_rate.interpolate(method='time', inplace=True, limit_direction='both')
hs_rate.columns = ['hs_rate']

# eth
eth = pd.read_excel('Data/ETH.xlsx', parse_dates=[0], index_col=0, date_parser=mydateparser, usecols='A,E')[start_date:end_date]
eth.index.name = index_name
eth.columns = ['eth']

# ggl_trends: only weekly data - missing values interpolated
# ggl_trends: weighted average of 15 countries
ggl_trends = pd.read_csv('Data/googletrends.txt', index_col=0)[start_date:end_date]
ggl_trends.index = pd.DatetimeIndex(ggl_trends.index, normalize=True).normalize()
ggl_trends.index.name = index_name
ggl_trends = ggl_trends.reindex(idx, fill_value=None)
ggl_trends.interpolate(method='time', inplace=True, limit_direction='both')
ggl_trends['btc-average'] = np.round(ggl_trends.sum(axis=1)/15)
ggl_trends = ggl_trends[['btc-average']].copy()
ggl_trends.columns = ['ggl_trends']

# wiki_views: sum of 99 countries
wiki_views = pd.read_excel('Data/wikipedia.xlsx', parse_dates=[0], index_col=0, date_parser=mydateparser1)[start_date:end_date]
wiki_views.index = pd.DatetimeIndex(wiki_views.index, normalize=True).normalize()
wiki_views.index.name = index_name
wiki_views['wiki-total'] = np.round(wiki_views.sum(axis=1))
wiki_views = wiki_views[['wiki-total']].copy()
wiki_views.columns = ['wiki_views']

# wti_oil
oil_wti = pd.read_excel('Data/DCOILWTICO.xls', parse_dates=[0], index_col=0, skiprows=10, date_parser=mydateparser)[start_date:end_date]
oil_wti.index = pd.DatetimeIndex(oil_wti.index, normalize=True).normalize()
oil_wti.index.name = index_name
oil_wti.columns = ['oil_wti']
oil_wti = oil_wti.loc[(oil_wti!=0).any(1)]

# gold
gold = pd.read_excel('Data/GOLDAMGBD228NLBM10AM.xls', parse_dates=[0], index_col=0, skiprows=10, date_parser=mydateparser)[start_date:end_date]
gold.index = pd.DatetimeIndex(gold.index, normalize=True).normalize()
gold.index.name = index_name
gold.columns = ['gold']
gold = gold.loc[(gold!=0).any(1)]

# sp500
sp500 = pd.read_excel('Data/SP500.xls', parse_dates=[0], index_col=0, skiprows=10, date_parser=mydateparser)[start_date:end_date]
sp500.index = pd.DatetimeIndex(sp500.index, normalize=True).normalize()
sp500.index.name = index_name
sp500.columns = ['sp500']
sp500 = sp500.loc[(sp500!=0).any(1)]

# sse 
sse = pd.read_excel('Data/SSEcomposite.xlsx', parse_dates=[0], index_col=0, header=0, date_parser=mydateparser)[start_date:end_date]
sse.index = pd.DatetimeIndex(sse.index, normalize=True).normalize()
sse.index.name = index_name
sse = sse[['Zuletzt']].copy()
sse.columns = ['sse']

# ffd_rate
ffd_rate = pd.read_excel('Data/DFF.xls', parse_dates=[0], index_col=0, skiprows=10, date_parser=mydateparser)[start_date:end_date]
ffd_rate.index = pd.DatetimeIndex(ffd_rate.index, normalize=True).normalize()
ffd_rate.index.name = index_name
ffd_rate.columns = ['ffd_rate']

In [5]:
# merge data to one df (inner join)
from functools import reduce
temp = [btc,tot_btc,hs_rate,eth,ggl_trends,wiki_views,usd_eur,oil_wti,
              gold,sp500,sse,ffd_rate]
df = reduce(lambda left, right: pd.merge(left, right, left_index=True, right_index=True), temp)
df

,btc,tot_btc,hs_rate,eth,ggl_trends,wiki_views,usd_eur,oil_wti,gold,sp500,sse,ffd_rate
date,,,,,,,,,,,,
2015-08-07,279.58,1.447762e+07,3.775640e+05,2.770000,2.0,15560,1.0958,43.87,1091.35,2077.57,3744.20,0.14
2015-08-10,264.47,1.448130e+07,3.687855e+05,0.708448,2.0,15113,1.0994,44.94,1094.80,2104.18,3928.42,0.14
2015-08-11,270.39,1.448498e+07,3.754390e+05,1.070000,2.0,15320,1.1042,43.11,1113.25,2084.07,3927.91,0.15
2015-08-12,266.38,1.448865e+07,3.820926e+05,1.220000,2.0,15255,1.1198,43.22,1116.80,2086.05,3886.32,0.15
2015-08-13,264.08,1.449229e+07,3.887461e+05,1.830000,2.0,23711,1.1144,42.27,1117.35,2083.39,3954.56,0.15
...,...,...,...,...,...,...,...,...,...,...,...,...
2020-06-18,9411.84,1.840797e+07,1.090229e+08,232.100000,10.0,16234,1.1216,38.79,1732.65,3115.34,2939.32,0.09
2020-06-19,9288.02,1.840862e+07,1.074749e+08,227.140000,10.0,15721,1.1189,39.72,1728.55,3097.74,2967.63,0.09
2020-06-22,9648.72,1.841192e+07,1.074657e+08,242.530000,10.0,16482,1.1260,40.60,1745.45,3117.86,2965.27,0.08


In [6]:
# split data in pre and post bubble 2018
pre_bubble_end_date = '2017-12-01'
post_bubble_start_date = '2018-02-01'

# pre bubble df
pre_df = df[:pre_bubble_end_date]

# post bubble df
post_df = df[post_bubble_start_date:]

In [7]:
# save btc_new.txt dataset with 1112 observations for use in M1-M3
def save_btcnew():
    btc_cropped = df['btc']
    btc_cropped.to_csv(path_or_buf=save_plot_to+'btc_new.txt')

# save_btcnew()

In [8]:
### RUN TO DEFINE ALL FUNCTIONS ###

# define overview-printing function of dataframe
def dates_overview(dataframe):
    print('start_date:\t', dataframe.index[0])
    print('end_date:\t', dataframe.index[-1])
    print('len df:\t\t', len(dataframe))
    
# define plotting function of dataframe
import matplotlib.dates as mdates
def plot_df(dataframe):
    fig_i, axs = plt.subplots(4,3, figsize=(9.5,4.15))
    for i, ax in enumerate(axs.flatten()):
        data = dataframe[dataframe.columns[i]]
        ax.plot(data, color='black')
        ax.set_title(dataframe.columns[i])
        ax.xaxis.set_ticks_position('none')
        ax.yaxis.set_ticks_position('none')
        ax.xaxis.set_major_formatter(mdates.DateFormatter("%Y-%m"))
        ax.xaxis.set_minor_formatter(mdates.DateFormatter("%Y-%m"))
        #ax.spines['top'].set_alpha(0)
        ax.tick_params(labelsize=8)
    fig_i.set_size_inches(12,5)
    fig_i.tight_layout()
    return fig_i

# define log-taking and relabeling function
def log_of_df(dataframe):
    df_log = np.log(dataframe)
    new_cols = list()
    for i in df_log.columns:
        new_cols.append(i+'_log')
    df_log.columns = new_cols
    # fill na value of negative oil price on 2020-04-20 with 0
    df_log.fillna(value=0, inplace=True)
    return df_log

# define correlation-table generating function of dataframe
def corr_table_aslatex_of_df(dataframe):
    corr = dataframe.corr().round(3)
    print(tabulate(corr, headers=corr.columns, showindex=True, tablefmt="latex"))

# define heatmap-generating function of dataframe
def heatmap_corr_of_df(dataframe):
    corr = dataframe.corr()
    # more html colors here: https://www.w3schools.com/colors/colors_names.asp
    # pal = sns.light_palette('lightgrey', as_cmap=True)
    ax = sns.heatmap(corr, xticklabels=corr.columns.values, yticklabels=corr.columns.values, annot=True, 
            annot_kws={'size':7}, vmin=-1, center=0, vmax=1, cmap="YlGnBu")
    bottom, top = ax.get_ylim()
    ax.set_ylim(bottom + 0.5, top - 0.5)
    fig_3 = plt.gcf()
    fig_3.set_size_inches(9.5,4.15)
    plt.tick_params(axis='both', which='major', labelsize=7.5, labelbottom = False, bottom=False, top = False, labeltop=True)
    plt.xticks(fontsize=7.5, rotation=1)
    plt.yticks(fontsize=7.5)
    
# define train/test-splitting function of dataframe
def split_traintest_df(dataframe):
    train_size = int(len(dataframe) * 0.8)
    df_train, df_test = dataframe[0:train_size], dataframe[train_size:]
    # create dataframe for printout
    data = {'Dataframe': ['dataframe', 'df_train', 'df_test'],
            'date_start': [dataframe.index[0], df_train.index[0], df_test.index[0]],
            'date_end': [dataframe.index[-1], df_train.index[-1], df_test.index[-1]],
            'nobs': [len(dataframe), len(df_train), len(df_test)]}
    df_print = pd.DataFrame(data, columns=['Dataframe','date_start','date_end','nobs'])
    print(df_print)
    return df_train, df_test

# define adf- and pp-testing of dataframe with latex-printout on/off
from arch.unitroot import ADF, PhillipsPerron
def stationarity_tests(dataframe, latex):
    for col in dataframe:
        adf = ADF(dataframe[col])
        pp = PhillipsPerron(dataframe[col])
        if latex is False:
            # write summary as plain text to std.out
            print('Timeseries:\t',col,'\n',
                  adf.summary(),'\n\n',pp.summary(),'\n\n\n')
        else:
            # write summary as latex to file
            with open(save_plot_to + 'Stationarity_Tests_LaTeX.txt', 'a') as myfile:
                myfile.write('Timeseries:\t'+col+'\n'
                         +adf.summary().as_latex()+'\n\n'
                         +pp.summary().as_latex()+'\n\n\n')

# define first difference-taking function of dataframe
def diff_of_df(dataframe):
    df_train_log_diff = df_train_log.diff()
    # relabel columns
    new_cols = list()
    for i in df_train_log_diff.columns:
        new_cols.append(i+'_diff')
    df_train_log_diff.columns = new_cols
    return df_train_log_diff

# define stationaritiy table-generating function of a dataframe filepath
def stationarity_table_aslatex_from_df(filepath):
    # read in dataframe from .txt file
    stationarity_dataframe = pd.read_csv(save_plot_to+filepath, delimiter='\s+', header=0)
    # print dataframe as latex output
    print(tabulate(stationarity_dataframe, headers=stationarity_dataframe.columns, showindex=False, tablefmt="latex"))

# define granger causality test performing function of a dataframe
# code taken from: https://stackoverflow.com/questions/58005681/is-it-possible-to-run-a-vector-autoregression-analysis-on-a-large-gdp-data-with
from statsmodels.tsa.stattools import grangercausalitytests
maxlag=15
test = 'ssr-chi2test'
def grangers_causality_matrix(X_train, variables, test = 'ssr_chi2test', verbose=False):
    dataset = pd.DataFrame(np.zeros((len(variables), len(variables))), columns=variables, index=variables)
    for c in dataset.columns:
        for r in dataset.index:
            test_result = grangercausalitytests(X_train[[r,c]], maxlag=maxlag, verbose=False)
            p_values = [round(test_result[i+1][0][test][1],4) for i in range(maxlag)]
            if verbose: print(f'Y = {r}, X = {c}, P Values = {p_values}')
            min_p_value = np.min(p_values)
            dataset.loc[r,c] = min_p_value
    dataset.columns = [var + '_x' for var in variables]
    dataset.index = [var + '_y' for var in variables]
    return dataset

# define granger causality table generating function from dataframe
def grangercausality_table_aslatex_of_df(dataframe):
    print(tabulate(dataframe, headers=dataframe.columns, showindex=True, tablefmt="latex"))

# define grangercausality heatmap generating function of a dataframe
def grangercausality_heatmap_of_df(dataframe):
    fig, ax = plt.subplots()
    sns.heatmap(dataframe, xticklabels=dataframe.columns.values, yticklabels=dataframe.index.values,
                 annot=True, annot_kws={'size':7}, vmin=0, vmax=1, cmap="YlGnBu_r", ax=ax)
    bottom, top = ax.get_ylim()
    ax.set_ylim(bottom + 0.5, top - 0.5)
    fig = plt.gcf()
    fig.set_size_inches(9.5,4.15)
    plt.tick_params(axis='both', which='major', labelsize=7, labelbottom = False, bottom=False, top = False, labeltop=True)
    plt.xticks(fontsize=7, rotation=45)
    plt.yticks(fontsize=7)
    plt.set_size_inches(12,5)
    plt.tight_layout()
    return fig

# define VAR-order selecting function of dataframe
from statsmodels.tsa.vector_ar.var_model import VAR
def select_VAR_order(dataframe):
    var_model = VAR(endog=dataframe)
    var_order_res = var_model.select_order(15).summary()
    res_as_html = var_order_res.as_html()
    df_var_order = pd.read_html(res_as_html, header=0, index_col=0)[0]
    return df_var_order

# define VAR-order latex-table-generating function of a var_order
def var_order_aslatex_of_order(var_order):
    print(tabulate(var_order, headers=var_order.columns, showindex=True, tablefmt="latex"))

# define VAR-model-fitting function of a dataframe and a lag order
def fit_VAR_model_of_df(dataframe, order):
    var_model = VAR(endog=dataframe)
    var_fitted = var_model.fit(maxlags = order)
    return var_fitted


# transform var_fitted into data frame
# code taken from: https://stackoverflow.com/questions/51734180/converting-statsmodels-summary-object-to-pandas-dataframe
def results_summary_to_dataframe(results):
    '''take the result of an statsmodel results table and transforms it into a dataframe'''
    pvals = results.pvalues
    coeff = results.params
    conf_lower = results.conf_int()[0]
    conf_higher = results.conf_int()[1]

    results_df = pd.DataFrame({"pvals":pvals,
                               "coeff":coeff,
                               "conf_lower":conf_lower,
                               "conf_higher":conf_higher
                                })
    #Reordering
    results_df = results_df[["coeff","pvals","conf_lower","conf_higher"]]
    return results_df

# define dataframe generating function from .txt file at filepath
def var_coeffs_aslatex_from_txt(filepath):
    # read in df from textfile
    var_btc_coefficients = pd.read_csv(save_plot_to+filepath, delimiter='\s+', header=0)
    var_btc_coefficients.set_index(var_btc_coefficients.columns[0], inplace=True)
    # generate latex output
    print(tabulate(var_btc_coefficients.round(4), headers=var_btc_coefficients.columns, showindex=True, tablefmt="latex"))

# define rolling forecast function
from sklearn.metrics import mean_squared_error as mse
def var_rolling_forecast(dataframe, lag_order):
    # create train and test dataframes
    train_size = int(len(dataframe) * 0.8)
    dataframe_train, dataframe_test = dataframe[0:train_size], dataframe[train_size:]
    index = len(dataframe_train) - 1
    # initialize lists
    pred_val_btc = list()
    lo_conf_int_btc = list()
    up_conf_int_btc = list()
    ausreisser_ctr = 0
    # rolling forecast
    for i in range(len(dataframe_test)):
        # input data, dependent on i
        input_data = log_of_df(dataframe).diff().values[lag_order+1:index + i]
        # fit model and predict 1 step
        dataframe_var = VAR(endog=input_data)
        dataframe_var_fitted = dataframe_var.fit(maxlags=lag_order)
        dataframe_var_result = dataframe_var_fitted.forecast_interval(y=input_data, steps=1, alpha=0.05)
        # obtain absolute (inversed) btc values for mean prediction, upper- and lower confidence interval
        yhat_btc = np.exp(dataframe_var_result[0][0][0] + np.log(dataframe).iloc[:,0][index+i])
        lo_conf_btc_val = np.exp(dataframe_var_result[1][0][0] + np.log(dataframe).iloc[:,0][index+i])
        up_conf_btc_val = np.exp(dataframe_var_result[2][0][0] + np.log(dataframe).iloc[:,0][index+i])
        if (dataframe.iloc[:,0][index+i+1] > up_conf_btc_val) or (dataframe.iloc[:,0][index+i+1] < lo_conf_btc_val):
            ausreisser_ctr += 1
        pred_val_btc.append(yhat_btc)
        lo_conf_int_btc.append(lo_conf_btc_val)
        up_conf_int_btc.append(up_conf_btc_val)
    # return [0]: mse, [1]: ausreisser ctr, [2]: predictions, [3]: lo_confint, [4]: up_confint
    return (np.sqrt(mse(pred_val_btc, dataframe_test.iloc[:,0].values)), ausreisser_ctr,
            pred_val_btc, lo_conf_int_btc, up_conf_int_btc)

# define series-generating and plotting function for dataframe_var_pred_result
def plot_var_pred_result(dataframe, dataframe_train, dataframe_test, dataframe_var_pred_result):
    # make series for plotting pred. vs. actual
    index_pred = np.arange(len(dataframe_train) + 1, len(dataframe) + 1)
    pred_val_btc_series = pd.Series(dataframe_var_pred_result[2], index=index_pred)
    test_series_btc = pd.Series(dataframe_test.iloc[:,0].values, index=index_pred)
    lo_conf_int_btc_series = pd.Series(dataframe_var_pred_result[3], index=index_pred)
    up_conf_int_btc_series = pd.Series(dataframe_var_pred_result[4], index=index_pred)
    # create plot: 
    fig, axs = plt.subplots(1,2)
    axs[0].plot(dataframe_train.iloc[:,0].values, label= r'$btc_T$', color='black')
    axs[0].plot(test_series_btc, label= r'$btc_{T+h}$', color='green')
    axs[0].plot(pred_val_btc_series, label= r'$\hat{btc}_{T+h}$', color= 'red')
    axs[0].fill_between(lo_conf_int_btc_series.index, lo_conf_int_btc_series, up_conf_int_btc_series, color='k', alpha=0.1)
    axs[0].legend(loc='upper left')
    axs[0].title.set_text('Gesamter Zeitraum')
    axs[1].plot(test_series_btc, label= r'$btc_{T+h}$', color='green')
    axs[1].plot(pred_val_btc_series, label= r'$\hat{btc}_{T+h}$', color= 'red')
    axs[1].fill_between(lo_conf_int_btc_series.index, lo_conf_int_btc_series, up_conf_int_btc_series, color='k', alpha=0.1)
    axs[1].legend(loc='upper left')
    axs[1].title.set_text('Vorhersage-Zeitraum')
    fig.set_size_inches(15,2.5)
    return fig

# define cointegration rank testing function based on select_coint_rank of a dataframe
from statsmodels.tsa.vector_ar.vecm import select_coint_rank
def print_select_coint_rank(dataframe, method, k_ar_diff, signif):
    dataframe_vecm_rank = select_coint_rank(dataframe,det_order=-1,k_ar_diff=k_ar_diff,method=method,signif=signif)
    print(dataframe_vecm_rank.summary())
    # summary output must be copied and saved as .txt

# define trace cointegration testing function for dataframe
from statsmodels.tsa.vector_ar.vecm import coint_johansen
def print_cointegration_test_of(dataframe, k_ar_diff, signif):
    result = coint_johansen(dataframe, -1, k_ar_diff)
    d = {'0.90':0, '0.95':1, '0.99':2}
    trace_stat = result.lr1
    crit_val_trace_stat = result.cvt[:, d[str(1-signif)]]
    
    def adjust(val, length = 6): 
        return str(val).ljust(length)
    
    print(' Variable  \t\ttest-statistic\tCV(95%)\t\tSignif\n')
    for col, trace, cvt in zip(dataframe.columns, trace_stat, crit_val_trace_stat):
        if col is 'dax' or col is 'sp500':
            tab = 2
        else:
            tab = 1
        print(adjust(col), '\t'*tab, adjust(round(trace,2), 9),'\t', adjust(cvt, 8), '\t' , trace > cvt)
    # -> Copy summary output and save as .txt
    
# define VECM specifying function for dataframe
from statsmodels.tsa.vector_ar.vecm import VECM
def get_vecm_model(dataframe,k_ar_diff,coint_rank,det):
    dataframe_vecm = VECM(endog=dataframe,k_ar_diff=k_ar_diff,coint_rank=coint_rank,deterministic=det)
    dataframe_vecm_fitted = dataframe_vecm.fit()
    return dataframe_vecm_fitted

# define rolling forecast function
from sklearn.metrics import mean_squared_error as mse
def vecm_rolling_forecast(dataframe, lag_order, coint_rank, det):
    # create train and test dataframes
    train_size = int(len(dataframe) * 0.8)
    dataframe_train, dataframe_test = dataframe[0:train_size], dataframe[train_size:]
    index = len(dataframe_train) - 1
    # initialize lists
    pred_val_btc = list()
    lo_conf_int_btc = list()
    up_conf_int_btc = list()
    ausreisser_ctr = 0
    # rolling forecast
    for i in range(len(dataframe_test)):
        # input data, dependent on i
        input_data = log_of_df(dataframe).values[lag_order:index + i]
        # fit model and predict 1 step
        dataframe_vecm_fitted = get_vecm_model(dataframe=input_data,k_ar_diff=lag_order,coint_rank=coint_rank,det=det)
        dataframe_vecm_result = dataframe_vecm_fitted.predict(steps=1,alpha=0.05)
        # obtain absolute (inversed) btc values for mean prediction, upper- and lower confidence interval
        yhat_btc = np.exp(dataframe_vecm_result[0][0][0])
        lo_conf_btc_val = np.exp(dataframe_vecm_result[1][0][0])
        up_conf_btc_val = np.exp(dataframe_vecm_result[2][0][0])
        if (dataframe.iloc[:,0][index+i+1] > up_conf_btc_val) or (dataframe.iloc[:,0][index+i+1] < lo_conf_btc_val):
            ausreisser_ctr += 1
        pred_val_btc.append(yhat_btc)
        lo_conf_int_btc.append(lo_conf_btc_val)
        up_conf_int_btc.append(up_conf_btc_val)
    # return [0]: mse, [1]: ausreisser ctr, [2]: predictions, [3]: lo_confint, [4]: up_confint
    return (np.sqrt(mse(pred_val_btc, dataframe_test.iloc[:,0].values)), ausreisser_ctr,
            pred_val_btc, lo_conf_int_btc, up_conf_int_btc)


## not used ##
# imports the coint_johansen function to test for cointegration as a prerequisite for VEC 
# modeling
from statsmodels.tsa.vector_ar.vecm import coint_johansen

# uses https://nbviewer.jupyter.org/github/mapsa/seminario-doc-2014/blob/master/cointegration-
# example.ipynb to create functions to return the number of cointegrating vectors based 
# on the Trace version if the Johansen Cointegration Test
def johansen_trace(y, p):
        N, l = y.shape
        joh_trace = coint_johansen(y, 0, p)
        r = 0
        for i in range(l):
            if joh_trace.lr1[i] > joh_trace.cvt[i, 1]:     # 0: 90%  1:95% 2: 99%
                r = i + 1
        joh_trace.r = r

        return joh_trace

# loops through 1 to 10 lags of trading days
#for i in range(1, 11): 
    # tests for cointegration at i lags
    #joh_trace = johansen_trace(df_select_train_log, i)
    # prints the results
    #print('Using the Trace Test, there are', joh_trace.r, '''cointegrating vectors at 
    #%s lags between the df_select_train_log''' % i)
    # prints a space for readability
    #print()

In [9]:
# overview of df
dates_overview(df)

start_date:	 2015-08-07 00:00:00
end_date:	 2020-06-24 00:00:00
len df:		 1112


In [10]:
# plot df
fig_1 = plot_df(df)

C:\Users\joche\Anaconda3\lib\site-packages\pandas\plotting\_matplotlib\converter.py:103: FutureWarning: Using an implicitly registered datetime converter for a matplotlib plotting method. The converter was registered by pandas on import. Future versions of pandas will require you to explicitly register matplotlib converters.

To register the converters:
	>>> from pandas.plotting import register_matplotlib_converters
	>>> register_matplotlib_converters()
  warnings.warn(msg, FutureWarning)


In [11]:
# take log of df
df_log = log_of_df(df)

C:\Users\joche\Anaconda3\lib\site-packages\ipykernel_launcher.py:29: RuntimeWarning: invalid value encountered in log


In [ ]:
# plot log time series
fig_2 = plot_df(df_log)

In [ ]:
# save fig_2
fig_2.savefig(save_plot_to+'M4_fig_2.svg',format='svg',bbox_inches='tight',pad_inches = 0)

In [12]:
# correlation matrix of df
corr_table_aslatex_of_df(df)

\begin{tabular}{lrrrrrrrrrrrr}
\hline
            &    btc &   tot\_btc &   hs\_rate &   eth &   ggl\_trends &   wiki\_views &   usd\_eur &   oil\_wti &   gold &   sp500 &    sse &   ffd\_rate \\
\hline
 btc        &  1     &     0.756 &     0.65  & 0.736 &        0.711 &        0.38  &     0.463 &     0.425 &  0.575 &   0.823 & -0.13  &      0.564 \\
 tot\_btc    &  0.756 &     1     &     0.881 & 0.361 &        0.33  &       -0.034 &     0.145 &     0.287 &  0.765 &   0.929 & -0.464 &      0.642 \\
 hs\_rate    &  0.65  &     0.881 &     1     & 0.144 &        0.157 &       -0.223 &    -0.068 &     0.046 &  0.831 &   0.832 & -0.463 &      0.392 \\
 eth        &  0.736 &     0.361 &     0.144 & 1     &        0.653 &        0.484 &     0.772 &     0.515 &  0.175 &   0.474 &  0.17  &      0.409 \\
 ggl\_trends &  0.711 &     0.33  &     0.157 & 0.653 &        1     &        0.799 &     0.469 &     0.236 &  0.233 &   0.377 &  0.186 &      0.277 \\
 wiki\_views &  0.38  &    -0.034 &    

In [13]:
# correlation heatmap of df
heatmap_corr_of_df(df)

In [14]:
# split df in df_train and df_test
df_train, df_test = split_traintest_df(df)

   Dataframe date_start   date_end  nobs
0  dataframe 2015-08-07 2020-06-24  1112
1   df_train 2015-08-07 2019-07-01   889
2    df_test 2019-07-02 2020-06-24   223


In [15]:
# take log of df_train
df_train_log = log_of_df(df_train)

In [ ]:
# plot df_train_log
fig_4 = plot_df(df_train_log)

In [16]:
# adf- and pp-testing of df_train_log - no latex printout
stationarity_tests(df_train_log, latex=False)

Timeseries:	 btc_log 
    Augmented Dickey-Fuller Results   
Test Statistic                 -0.883
P-value                         0.794
Lags                                0
-------------------------------------

Trend: Constant
Critical Values: -3.44 (1%), -2.86 (5%), -2.57 (10%)
Null Hypothesis: The process contains a unit root.
Alternative Hypothesis: The process is weakly stationary. 

      Phillips-Perron Test (Z-tau)    
Test Statistic                 -0.931
P-value                         0.778
Lags                               21
-------------------------------------

Trend: Constant
Critical Values: -3.44 (1%), -2.86 (5%), -2.57 (10%)
Null Hypothesis: The process contains a unit root.
Alternative Hypothesis: The process is weakly stationary. 



Timeseries:	 tot_btc_log 
    Augmented Dickey-Fuller Results   
Test Statistic                 -2.824
P-value                         0.055
Lags                               19
-------------------------------------

Trend: Constan

In [17]:
# take first difference of df_train_log
df_train_log_diff = diff_of_df(df_train_log)
df_train_log_diff

,btc_log_diff,tot_btc_log_diff,hs_rate_log_diff,eth_log_diff,ggl_trends_log_diff,wiki_views_log_diff,usd_eur_log_diff,oil_wti_log_diff,gold_log_diff,sp500_log_diff,sse_log_diff,ffd_rate_log_diff
date,,,,,,,,,,,,
2015-08-07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015-08-10,-0.055561,0.000254,-0.023525,-1.363526,0.000000,-0.029148,0.003280,0.024098,0.003156,0.012727,0.048029,0.000000
2015-08-11,0.022138,0.000254,0.017881,0.412337,0.000000,0.013604,0.004357,-0.041573,0.016712,-0.009603,-0.000130,0.068993
2015-08-12,-0.014942,0.000254,0.017567,0.131192,0.000000,-0.004252,0.014029,0.002548,0.003184,0.000950,-0.010645,0.000000
2015-08-13,-0.008672,0.000251,0.017264,0.405465,0.000000,0.441032,-0.004834,-0.022226,0.000492,-0.001276,0.017407,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...
2019-06-25,0.068426,0.000099,0.024530,0.024534,-0.083382,-0.007638,-0.000615,-0.001734,0.017002,-0.009542,-0.008708,0.000000
2019-06-26,0.098867,0.000098,0.023942,0.056881,-0.044452,0.413610,0.000351,0.026371,-0.016078,-0.001235,-0.001943,0.000000
2019-06-27,-0.151819,0.000098,0.010964,-0.134843,-0.046520,-0.173980,-0.001582,0.000169,-0.003204,0.003816,0.006868,0.000000


In [ ]:
# plot df_train_log_diff
fig_5 = plot_df(df_train_log_diff[1:])

In [ ]:
# save fig_5
fig_5.savefig(save_plot_to+'M4_fig_5.svg',format='svg',bbox_inches='tight',pad_inches = 0)

In [18]:
# adf- and pp-testing of df_train_log_diff - no latex printout
stationarity_tests(df_train_log_diff[1:], latex=False)
    

Timeseries:	 btc_log_diff 
    Augmented Dickey-Fuller Results   
Test Statistic                -29.318
P-value                         0.000
Lags                                0
-------------------------------------

Trend: Constant
Critical Values: -3.44 (1%), -2.86 (5%), -2.57 (10%)
Null Hypothesis: The process contains a unit root.
Alternative Hypothesis: The process is weakly stationary. 

      Phillips-Perron Test (Z-tau)    
Test Statistic                -29.679
P-value                         0.000
Lags                               21
-------------------------------------

Trend: Constant
Critical Values: -3.44 (1%), -2.86 (5%), -2.57 (10%)
Null Hypothesis: The process contains a unit root.
Alternative Hypothesis: The process is weakly stationary. 



Timeseries:	 tot_btc_log_diff 
    Augmented Dickey-Fuller Results   
Test Statistic                 -2.820
P-value                         0.055
Lags                               18
-------------------------------------

Tren

In [ ]:
# plot stationarity table of df
stationarity_table_aslatex_from_df('M4_stationarity tests.txt')

In [19]:
# perform granger causality test each on each based on df_train_log_diff
granger_causality_matrix = grangers_causality_matrix(df_train_log_diff[1:], variables = df_train_log_diff.columns)
granger_causality_matrix.round(4)

,btc_log_diff_x,tot_btc_log_diff_x,hs_rate_log_diff_x,eth_log_diff_x,ggl_trends_log_diff_x,wiki_views_log_diff_x,usd_eur_log_diff_x,oil_wti_log_diff_x,gold_log_diff_x,sp500_log_diff_x,sse_log_diff_x,ffd_rate_log_diff_x
btc_log_diff_y,1.0000,0.5272,0.0225,0.0784,0.0055,0.2011,0.2354,0.4655,0.3461,0.1532,0.6332,0.4867
tot_btc_log_diff_y,0.6692,1.0000,0.1870,0.0567,0.1614,0.0000,0.0362,0.0008,0.4126,0.0181,0.0009,0.0000
hs_rate_log_diff_y,0.0850,0.0000,1.0000,0.0223,0.0467,0.0797,0.0150,0.0029,0.0229,0.1382,0.4550,0.1592
eth_log_diff_y,0.0038,0.1471,0.1155,1.0000,0.4148,0.1589,0.0975,0.5212,0.2341,0.1532,0.0184,0.0384
ggl_trends_log_diff_y,0.0750,0.0225,0.1172,0.2559,1.0000,0.0050,0.0417,0.1165,0.1172,0.1349,0.4080,0.0079
wiki_views_log_diff_y,0.0001,0.1781,0.1756,0.4502,0.0000,1.0000,0.4715,0.0284,0.1477,0.1051,0.0831,0.0207
usd_eur_log_diff_y,0.2558,0.0974,0.5620,0.0306,0.1226,0.5229,1.0000,0.0175,0.1499,0.2106,0.0364,0.0024
oil_wti_log_diff_y,0.0331,0.1157,0.1933,0.0012,0.0271,0.6849,0.2089,1.0000,0.3901,0.0051,0.0368,0.0561
gold_log_diff_y,0.5008,0.0778,0.1955,0.0814,0.3031,0.7110,0.0000,0.0077,1.0000,0.0259,0.0947,0.1240
sp500_log_diff_y,0.1098,0.4486,0.0199,0.0004,0.0204,0.0668,0.1182,0.4076,0.1643,1.0000,0.0000,0.0002


In [ ]:
# generate granger causality table as latex output
grangercausality_table_aslatex_of_df(granger_causality_matrix)

In [ ]:
# generate granger causality heatmap of df_train_log_diff
fig_6 = grangercausality_heatmap_of_df(granger_causality_matrix)

In [ ]:
# save fig_6
fig_6.savefig(save_plot_to+'M4_fig_6.svg',format='svg',bbox_inches='tight',pad_inches = 0)

In [ ]:
## VAR: df_select_train_log_diff (3 Variables) ##

In [20]:
# based on granger-causality: new df with btc, hs_rate and ggl_trends
df_select_train_log_diff = df_train_log_diff[['btc_log_diff','hs_rate_log_diff','ggl_trends_log_diff']]
df_select_train_log_diff

,btc_log_diff,hs_rate_log_diff,ggl_trends_log_diff
date,,,
2015-08-07,NaN,NaN,NaN
2015-08-10,-0.055561,-0.023525,0.000000
2015-08-11,0.022138,0.017881,0.000000
2015-08-12,-0.014942,0.017567,0.000000
2015-08-13,-0.008672,0.017264,0.000000
...,...,...,...
2019-06-25,0.068426,0.024530,-0.083382
2019-06-26,0.098867,0.023942,-0.044452
2019-06-27,-0.151819,0.010964,-0.046520


In [21]:
# select VAR-order for df_select_train_log_diff: 5
df_select_var_order = select_VAR_order(df_select_train_log_diff[1:])
df_select_var_order

C:\Users\joche\Anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:583: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)


,AIC,BIC,FPE,HQIC
0,-18.94,-18.92,5.952e-09,-18.93
1,-19.03,-18.96*,5.437e-09,-19.00*
2,-19.04,-18.92,5.388e-09,-19.00
3,-19.04,-18.87,5.401e-09,-18.97
4,-19.05,-18.84,5.318e-09,-18.97
5,-19.05,-18.79,5.340e-09,-18.95
6,-19.06*,-18.75,5.271e-09*,-18.94
7,-19.05,-18.69,5.350e-09,-18.91
8,-19.03,-18.62,5.432e-09,-18.87
9,-19.04,-18.58,5.364e-09,-18.87


In [22]:
# generate var_select_order latex output from df_select_var_order
var_order_aslatex_of_order(df_select_var_order)

\begin{tabular}{rllll}
\hline
    & AIC     & BIC     & FPE        & HQIC    \\
\hline
  0 & -18.94  & -18.92  & 5.952e-09  & -18.93  \\
  1 & -19.03  & -18.96* & 5.437e-09  & -19.00* \\
  2 & -19.04  & -18.92  & 5.388e-09  & -19.00  \\
  3 & -19.04  & -18.87  & 5.401e-09  & -18.97  \\
  4 & -19.05  & -18.84  & 5.318e-09  & -18.97  \\
  5 & -19.05  & -18.79  & 5.340e-09  & -18.95  \\
  6 & -19.06* & -18.75  & 5.271e-09* & -18.94  \\
  7 & -19.05  & -18.69  & 5.350e-09  & -18.91  \\
  8 & -19.03  & -18.62  & 5.432e-09  & -18.87  \\
  9 & -19.04  & -18.58  & 5.364e-09  & -18.87  \\
 10 & -19.03  & -18.52  & 5.455e-09  & -18.83  \\
 11 & -19.03  & -18.47  & 5.430e-09  & -18.82  \\
 12 & -19.02  & -18.42  & 5.472e-09  & -18.79  \\
 13 & -19.02  & -18.37  & 5.483e-09  & -18.77  \\
 14 & -19.01  & -18.30  & 5.550e-09  & -18.74  \\
 15 & -18.99  & -18.24  & 5.639e-09  & -18.71  \\
\hline
\end{tabular}


In [24]:
# obtain VAR-model for df_select_train_log_diff and lag order = 5
df_select_var = fit_VAR_model_of_df(df_select_train_log_diff[1:], 5)
df_select_var.summary()
# -> Copy summary output and save as .txt

C:\Users\joche\Anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:583: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)


  Summary of Regression Results   
Model:                         VAR
Method:                        OLS
Date:           Thu, 01, Oct, 2020
Time:                     00:20:58
--------------------------------------------------------------------
No. of Equations:         3.00000    BIC:                   -18.7861
Nobs:                     883.000    HQIC:                  -18.9467
Log likelihood:           4698.11    FPE:                5.35016e-09
AIC:                     -19.0462    Det(Omega_mle):     5.06956e-09
--------------------------------------------------------------------
Results for equation btc_log_diff
                            coefficient       std. error           t-stat            prob
-----------------------------------------------------------------------------------------
const                          0.002733         0.001838            1.487           0.137
L1.btc_log_diff               -0.010046         0.034290           -0.293           0.770
L1.hs_rate_log_di

In [21]:
# read in and plot var_select_coeffs from textfile as latex: 'M4_df_select_VAR_coefficients.txt'
var_coeffs_aslatex_from_txt('M4_df_select_VAR_coefficients.txt')

\begin{tabular}{lrrrr}
\hline
                        &   coefficient &   std.error &   t-stat &   prob \\
\hline
 const                  &        0.005  &      0.0007 &    6.764 &  0     \\
 L1.btc\_log\_diff        &        0.002  &      0.0138 &    0.148 &  0.882 \\
 L1.hs\_rate\_log\_diff    &        0.272  &      0.0338 &    8.044 &  0     \\
 L1.ggl\_trends\_log\_diff &       -0.0056 &      0.0087 &   -0.639 &  0.523 \\
 L2.btc\_log\_diff        &        0.0318 &      0.0138 &    2.31  &  0.021 \\
 L2.hs\_rate\_log\_diff    &        0.0627 &      0.0347 &    1.806 &  0.071 \\
 L2.ggl\_trends\_log\_diff &       -0.0113 &      0.0087 &   -1.295 &  0.195 \\
 L3.btc\_log\_diff        &        0.0082 &      0.0139 &    0.592 &  0.554 \\
 L3.hs\_rate\_log\_diff    &       -0.0321 &      0.0347 &   -0.925 &  0.355 \\
 L3.ggl\_trends\_log\_diff &       -0.0159 &      0.0088 &   -1.814 &  0.07  \\
 L4.btc\_log\_diff        &       -0.0037 &      0.0139 &   -0.266 &  0.79  \\
 L4.hs\_rate\

In [25]:
# rolling forecast: obtain mse, ausreisser, predictions, lo-& up confint for df_select with lag order = 5
df_select_var_pred_result = var_rolling_forecast(df[['btc','hs_rate','ggl_trends']], 5)

# printout results
print('mse_select:\t\t', df_select_var_pred_result[0])
print('ausreisser_select:\t', df_select_var_pred_result[1])

mse_select:		 439.1335176962215
ausreisser_select:	 14


In [ ]:
# rolling forecast: create series and plot them for df_var_pred_result
fig_7 = plot_var_pred_result(df[['btc','hs_rate','ggl_trends']],
                             df_train[['btc','hs_rate','ggl_trends']],
                             df_test[['btc','hs_rate','ggl_trends']],
                             df_select_var_pred_result)

In [ ]:
# save fig_7
fig_7.savefig(save_plot_to+'M4_fig_7.svg',format='svg',bbox_inches='tight',pad_inches = 0)

In [ ]:
## VAR: df_train_log_diff (12 Variables) ##

In [26]:
# select VAR-order for df_train_log_diff: 5
df_var_order = select_VAR_order(df_train_log_diff[1:])
df_var_order

C:\Users\joche\Anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:583: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)


,AIC,BIC,FPE,HQIC
0,-95.56,-95.49*,3.163e-42,-95.53
1,-95.89,-95.04,2.261e-42,-95.57*
2,-95.84,-94.20,2.375e-42,-95.22
3,-95.84,-93.41,2.381e-42,-94.91
4,-95.86,-92.64,2.344e-42,-94.63
5,-95.97*,-91.96,2.108e-42*,-94.43
6,-95.85,-91.06,2.377e-42,-94.02
7,-95.76,-90.19,2.599e-42,-93.63
8,-95.67,-89.31,2.858e-42,-93.24
9,-95.59,-88.44,3.103e-42,-92.86


In [ ]:
# generate var_order latex output from df_var_order
var_order_aslatex_of_order(df_var_order)

In [27]:
# obtain VAR-model for df_train_log_diff and lag order = 5
df_var = fit_VAR_model_of_df(df_train_log_diff[1:], 5)
df_var.summary()
# -> Copy summary output and save as .txt

C:\Users\joche\Anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:583: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)


  Summary of Regression Results   
Model:                         VAR
Method:                        OLS
Date:           Thu, 01, Oct, 2020
Time:                     00:21:14
--------------------------------------------------------------------
No. of Equations:         12.0000    BIC:                   -91.7375
Nobs:                     883.000    HQIC:                  -94.1867
Log likelihood:           27949.7    FPE:                2.74113e-42
AIC:                     -95.7028    Det(Omega_mle):     1.22969e-42
--------------------------------------------------------------------
Results for equation btc_log_diff
                            coefficient       std. error           t-stat            prob
-----------------------------------------------------------------------------------------
const                          0.001063         0.003756            0.283           0.777
L1.btc_log_diff                0.011233         0.039179            0.287           0.774
L1.tot_btc_log_di

In [28]:
# read in and plot var_coeffs from textfile: 'M4_df_VAR_coefficients.txt'
var_coeffs_aslatex_from_txt('M4_df_VAR_coefficients.txt')

FileNotFoundError: [Errno 2] File b'C:\\\\Users\\joche\\OneDrive\\03 TUM - TUM-BWL\\Semester 8\\01 Bachelorarbeit\\04 Results\\Plots/M4_df_VAR_coefficients.txt' does not exist: b'C:\\\\Users\\joche\\OneDrive\\03 TUM - TUM-BWL\\Semester 8\\01 Bachelorarbeit\\04 Results\\Plots/M4_df_VAR_coefficients.txt'

In [19]:
# IRF: impulse response of df_var
from statsmodels.tsa.vector_ar.irf import IRAnalysis
df_var_irf = df_var.irf(15)
fig_9 = df_var_irf.plot(orth=False, impulse = 'btc_log_diff', response='wiki_views_log_diff', figsize=(8,15),subplot_params={'fontsize' : 9})
fig_9.tight_layout()

In [ ]:
# save fig_9
fig_9.savefig(save_plot_to+'M4_fig_9.svg',format='svg',bbox_inches='tight',pad_inches = 0)

In [20]:
# IRF: cumulative impulse responses of df_var
fig_10 = df_var_irf.plot_cum_effects(orth=False, impulse = 'btc_log_diff', response='wiki_views_log_diff',figsize=(8,15),subplot_params={'fontsize' : 9})
fig_10.tight_layout()

In [ ]:
# save fig_10
fig_10.savefig(save_plot_to+'M4_fig_10.svg',format='svg',bbox_inches='tight',pad_inches = 0)

In [29]:
# rolling forecast: obtain mse, ausreisser, predictions, lo-& up confint for df with lag order = 5
df_var_pred_result = var_rolling_forecast(df, 5)
# printout results
print('mse:\t\t', df_var_pred_result[0])
print('ausreisser:\t', df_var_pred_result[1])

C:\Users\joche\Anaconda3\lib\site-packages\ipykernel_launcher.py:29: RuntimeWarning: invalid value encountered in log
C:\Users\joche\Anaconda3\lib\site-packages\ipykernel_launcher.py:210: RuntimeWarning: invalid value encountered in log
C:\Users\joche\Anaconda3\lib\site-packages\ipykernel_launcher.py:211: RuntimeWarning: invalid value encountered in log
C:\Users\joche\Anaconda3\lib\site-packages\ipykernel_launcher.py:212: RuntimeWarning: invalid value encountered in log


mse:		 507.96568585996687
ausreisser:	 17


In [ ]:
# rolling forecast: create series and plot them for df_var_pred_result
fig_8 = plot_var_pred_result(df,df_train,df_test,df_var_pred_result)

In [ ]:
# save fig_8
fig_8.savefig(save_plot_to+'M4_fig_8.svg',format='svg',bbox_inches='tight',pad_inches = 0)

In [ ]:
## VECM: df_select_train_log (3 Variables) ##

In [ ]:
## ARCHIVE ##
# series and lag plot to show trend and autocorrelation
fig, axs = plt.subplots(2,1)
axs[0].plot(df_log)
pd.plotting.lag_plot(df_log, ax = axs[1])


In [30]:
# based on granger-causality: new df with btc, hs_rate and ggl_trends
df_select_train_log = df_train_log[['btc_log','hs_rate_log','ggl_trends_log']]

In [31]:
# determine cointegration rank of df_select_train_log with trace statistic
print_select_coint_rank(df_select_train_log,'trace',10,0.05)
# -> Copy summary output and save as .txt

Johansen cointegration test using trace test statistic with 5% significance level
r_0 r_1 test statistic critical value
-------------------------------------
  0   3          57.72          24.28
  1   3          28.34          12.32
  2   3          3.164          4.130
-------------------------------------


In [ ]:
# read in and plot from textfile as latex: 'M4_select_tab_10_cointrank_trace.txt'
var_coeffs_aslatex_from_txt('M4_select_tab_10_cointrank_trace.txt')
# -> Copy latex output and input in overleaf

In [32]:
# determine cointegration rank of df_select_train_log with maxeig statistic
print_select_coint_rank(df_select_train_log,'maxeig',10,0.05)
# -> Copy summary output and save as .txt

Johansen cointegration test using maximum eigenvalue test statistic with 5% significance level
r_0 r_1 test statistic critical value
-------------------------------------
  0   1          29.38          17.80
  1   2          25.17          11.22
  2   3          3.164          4.130
-------------------------------------


In [ ]:
# read in and plot from textfile as latex: 'M4_select_tab_11_cointrank_maxeig.txt'
var_coeffs_aslatex_from_txt('M4_select_tab_11_cointrank_maxeig.txt')
# -> Copy latex output and input in overleaf

In [33]:
# print out johansen cointegration result for df_select_train_log
print_cointegration_test_of(df_select_train_log,10,0.05)
# -> Copy summary output and save as .txt

 Variable  		test-statistic	CV(95%)		Signif

btc_log 	 57.72     	 24.2761  	 True
hs_rate_log 	 28.34     	 12.3212  	 True
ggl_trends_log 	 3.16      	 4.1296   	 False


In [ ]:
# read in and plot from textfile as latex: 'M4_select_tab_12_johansen_cointrank_trace.txt'
var_coeffs_aslatex_from_txt('M4_select_tab_12_johansen_cointrank_trace.txt')
# -> Copy latex output and input in overleaf

In [34]:
# obtain VECM(4) for df_select_train_log
df_select_vecm = get_vecm_model(df_select_train_log,4,2,'nc')
df_select_vecm.summary()
# -> Copy summary output and save as .txt

C:\Users\joche\Anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:583: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)


,coef,std err,z,P>|z|,[0.025,0.975]
L1.btc_log,-0.0004,0.034,-0.012,0.991,-0.066,0.066
L1.hs_rate_log,0.0893,0.082,1.090,0.276,-0.071,0.250
L1.ggl_trends_log,0.0187,0.022,0.863,0.388,-0.024,0.061
L2.btc_log,0.0270,0.034,0.800,0.424,-0.039,0.093
L2.hs_rate_log,-0.0065,0.085,-0.077,0.939,-0.173,0.160
L2.ggl_trends_log,0.0345,0.022,1.592,0.111,-0.008,0.077
L3.btc_log,0.0270,0.034,0.796,0.426,-0.039,0.093
L3.hs_rate_log,-0.0781,0.085,-0.923,0.356,-0.244,0.088
L3.ggl_trends_log,0.0156,0.022,0.715,0.475,-0.027,0.058
L4.btc_log,0.0472,0.034,1.390,0.165,-0.019,0.114


In [29]:
# read in and plot from textfile as latex: 'M4_df_select_VECM_coefficients.txt'
var_coeffs_aslatex_from_txt('M4_df_select_VECM_coefficients.txt')
# -> Copy latex output and input in overleaf

\begin{tabular}{lrrrrrr}
\hline
                   &    coef &   std.err &      z &   P\ensuremath{>}|z| &   [0.025 &   0.975] \\
\hline
 L1.btc\_log        & -0.0004 &     0.034 & -0.012 &   0.991 &   -0.066 &    0.066 \\
 L1.hs\_rate\_log    &  0.0893 &     0.082 &  1.09  &   0.276 &   -0.071 &    0.25  \\
 L1.ggl\_trends\_log &  0.0187 &     0.022 &  0.863 &   0.388 &   -0.024 &    0.061 \\
 L2.btc\_log        &  0.027  &     0.034 &  0.8   &   0.424 &   -0.039 &    0.093 \\
 L2.hs\_rate\_log    & -0.0065 &     0.085 & -0.077 &   0.939 &   -0.173 &    0.16  \\
 L2.ggl\_trends\_log &  0.0345 &     0.022 &  1.592 &   0.111 &   -0.008 &    0.077 \\
 L3.btc\_log        &  0.027  &     0.034 &  0.796 &   0.426 &   -0.039 &    0.093 \\
 L3.hs\_rate\_log    & -0.0781 &     0.085 & -0.923 &   0.356 &   -0.244 &    0.088 \\
 L3.ggl\_trends\_log &  0.0156 &     0.022 &  0.715 &   0.475 &   -0.027 &    0.058 \\
 L4.btc\_log        &  0.0472 &     0.034 &  1.39  &   0.165 &   -0.019 &    0.114 

In [35]:
# rolling forecast: obtain mse, ausreisser, predictions, lo-& up confint for df_select with lag order = 4
df_select_vecm_pred_result = vecm_rolling_forecast(df[['btc','hs_rate','ggl_trends']],lag_order=4,coint_rank=2,det='nc')

# printout results
print('mse_select:\t\t', df_select_vecm_pred_result[0])
print('ausreisser_select:\t', df_select_vecm_pred_result[1])

mse_select:		 628.8598662521392
ausreisser_select:	 29


In [ ]:
# rolling forecast: create series and plot them for df_vecm_pred_result
fig_11 = plot_var_pred_result(df[['btc','hs_rate','ggl_trends']],
                             df_train[['btc','hs_rate','ggl_trends']],
                             df_test[['btc','hs_rate','ggl_trends']],
                             df_select_vecm_pred_result)

In [ ]:
# save fig_11
fig_11.savefig(save_plot_to+'M4_fig_11.svg',format='svg',bbox_inches='tight',pad_inches = 0)

In [ ]:
## VECM: df_train_log (12 Variables) ##

In [36]:
# determine cointegration rank of df_train_log with trace statistic
print_select_coint_rank(df_train_log,'trace',10,0.05)
# -> Copy summary output and save as .txt

Johansen cointegration test using trace test statistic with 5% significance level
r_0 r_1 test statistic critical value
-------------------------------------
  0  12          441.3          311.1
  1  12          340.1          263.3
  2  12          272.8          219.4
  3  12          209.1          179.5
  4  12          158.2          143.7
  5  12          117.5          111.8
  6  12          79.60          83.94
-------------------------------------


In [ ]:
# read in and plot from textfile as latex: 'M4_tab_7_cointrank_trace.txt'
var_coeffs_aslatex_from_txt('M4_tab_7_cointrank_trace.txt')
# -> Copy latex output and input in overleaf

In [37]:
# determine cointegration rank of df_train_log with maxeig statistic
print_select_coint_rank(df_train_log,'maxeig',10,0.05)
# -> Copy summary output and save as .txt

Johansen cointegration test using maximum eigenvalue test statistic with 5% significance level
r_0 r_1 test statistic critical value
-------------------------------------
  0   1          101.2          73.09
  1   2          67.37          67.08
  2   3          63.66          61.04
  3   4          50.86          54.96
-------------------------------------


In [ ]:
# read in and plot from textfile as latex: 'M4_tab_8_cointrank_maxeig.txt'
var_coeffs_aslatex_from_txt('M4_tab_8_cointrank_maxeig.txt')
# -> Copy latex output and input in overleaf

In [38]:
# print out johansen cointegration result for df_train_log
print_cointegration_test_of(df_train_log,10,0.05)
# -> Copy summary output and save as .txt

 Variable  		test-statistic	CV(95%)		Signif

btc_log 	 441.34    	 311.1288 	 True
tot_btc_log 	 340.14    	 263.2603 	 True
hs_rate_log 	 272.77    	 219.4051 	 True
eth_log 	 209.11    	 179.5199 	 True
ggl_trends_log 	 158.24    	 143.6691 	 True
wiki_views_log 	 117.47    	 111.7797 	 True
usd_eur_log 	 79.6      	 83.9383  	 False
oil_wti_log 	 47.01     	 60.0627  	 False
gold_log 	 29.19     	 40.1749  	 False
sp500_log 	 15.96     	 24.2761  	 False
sse_log 	 6.11      	 12.3212  	 False
ffd_rate_log 	 0.0       	 4.1296   	 False


In [ ]:
# read in and plot from textfile as latex: 'M4_tab_9_johansen_cointrank_trace.txt'
var_coeffs_aslatex_from_txt('M4_tab_9_johansen_cointrank_trace.txt')
# -> Copy latex output and input in overleaf

In [39]:
# obtain vecm(5-1) for df_train_log and coint_rank 6
df_vecm = get_vecm_model(df_train_log,k_ar_diff=4,coint_rank=6,det='nc')
df_vecm.summary()

C:\Users\joche\Anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:583: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)


,coef,std err,z,P>|z|,[0.025,0.975]
L1.btc_log,0.0237,0.037,0.633,0.527,-0.050,0.097
L1.tot_btc_log,-10.4730,8.627,-1.214,0.225,-27.382,6.436
L1.hs_rate_log,0.1341,0.087,1.534,0.125,-0.037,0.306
L1.eth_log,-0.0114,0.022,-0.525,0.600,-0.054,0.031
L1.ggl_trends_log,0.0143,0.023,0.634,0.526,-0.030,0.059
L1.wiki_views_log,-0.0087,0.011,-0.777,0.437,-0.031,0.013
L1.usd_eur_log,-0.3145,0.329,-0.956,0.339,-0.959,0.330
L1.oil_wti_log,-0.0624,0.072,-0.870,0.384,-0.203,0.078
L1.gold_log,-0.0400,0.223,-0.180,0.857,-0.476,0.396
L1.sp500_log,0.3312,0.202,1.637,0.102,-0.065,0.728


In [ ]:
# read in and plot from textfile as latex: 'M4_tab_13_vecm_coeffs.txt'
var_coeffs_aslatex_from_txt('M4_tab_13_vecm_coeffs.txt')
# -> Copy latex output and input in overleaf

In [ ]:
# read in and plot from textfile as latex: 'M4_tab_14_vecm_alpha1.txt'
var_coeffs_aslatex_from_txt('M4_tab_14_vecm_alpha1.txt')
# -> Copy latex output and input in overleaf

In [ ]:
# read in and plot from textfile as latex: 'M4_tab_15_vecm_beta1.txt'
var_coeffs_aslatex_from_txt('M4_tab_15_vecm_beta1.txt')
# -> Copy latex output and input in overleaf

In [ ]:
# IRF: impulse response of df_vecm
df_vecm_irf = df_vecm.irf(15)
fig_12 = df_vecm_irf.plot(orth=False, response='btc_log', figsize=(8,15),subplot_params={'fontsize' : 9})
fig_12.tight_layout()

In [ ]:
# save fig_12
fig_12.savefig(save_plot_to+'M4_fig_12.svg',format='svg',bbox_inches='tight',pad_inches = 0)